In [1]:
import numpy as np
import sys
import tensorflow
from sklearn.metrics.pairwise import cosine_similarity as cosine_sim_vectors
from bert_serving.client import BertClient
import pandas as pd
import os
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn.cluster as clus
from sklearn.preprocessing import MinMaxScaler
bc = BertClient()

In [172]:
# np.set_printoptions(formatter={'float_kind':'{:f}'.format},threshold=sys.maxsize)
# test_vec = bc.encode(['court'])
# test_vec1 = bc.encode(['perfume'])
# bc.status

In [171]:
# vec1 = bc.encode(['he studies humanities'])
# vec2 = bc.encode(['the professor teaches humanities'])
# cos_sim=cosine_sim_vectors(vec1.reshape(1,-1), vec2.reshape(1,-1)) [0][0]
# print(cos_sim)

In [170]:
# a = np.array([1,2,3,4,5])
# b = np.array([1,2,3,4,5])
# sam_df = pd.DataFrame(a.reshape(-1, len(a)))
# sam_df = sam_df.append(pd.DataFrame(b.reshape(-1, len(b))))
# print(sam_df)

In [148]:
# os.chdir(r'C:/Users/ssund/Documents/adm_project1/')
# with BertClient() as bc:
#     out_df = pd.read_csv('C:/Users/ssund/Documents/adm_project1/sample.txt', names = ['file_name', 'wiki_word', 'index'])
#     index_df = pd.DataFrame({'Index' : out_df['index']})
#     vector_df = pd.DataFrame()
#     for i in out_df['wiki_word']:
#         word = i
#         vec = bc.encode([str(word)])
#         vector_df = vector_df.append(pd.DataFrame(vec[0].reshape(-1, len(vec[0]))))
#         print(vec)
#         high_vec = vec.max()
#         vector_df = vector_df.append(str(vec), ignore_index = True)
#         print(str(i)+' - '+str(high_vec))
#     print(vector_df)
#     print(vec[0])
#     final_df = index_df.join(vector_df)
#     print(final_df)

In [168]:
## word embedding of a sentence
# sent_vec = bc.encode(['This is a sentence'])
# print(sent_vec[0])
# test_df = pd.DataFrame(sent_vec[0].reshape(-1, len(sent_vec[0])))
# print(test_df)

### Get word embeddings for wiki words with context 

In [ ]:
os.chdir(r'C:/Users/ssund/Documents/adm_project1/')
# out_df = pd.read_csv('C:/Users/ssund/Documents/adm_project1/sample.txt', names = ['file_name', 'wiki_word', 'index'])
out_df = pd.read_csv('C:/Users/ssund/Documents/adm_project1/sentence_output.txt', names = ['file_name','sentence','wiki_word'])
vector_df = pd.DataFrame()
for i in out_df['sentence']:
    word = i
    vec = bc.encode([str(word)])
    vector_df = vector_df.append(pd.DataFrame(vec[0].reshape(-1, len(vec[0]))))
print(vector_df)

C:\Users\ssund\Anaconda3\lib\site-packages\bert_serving\client\__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


### Converting 'out.txt' to excel file

In [ ]:
vector_df.to_excel('C:/Users/ssund/Documents/adm_project1/kmeans_dataset.xlsx', 'Sheet1', index = False)

In [21]:
# vector_df.unique()

### Optimal number of clusters for k-means

In [ ]:
mms = MinMaxScaler()
mms.fit(vector_df)
data_transformed = mms.transform(vector_df)

In [ ]:
Sum_of_squared_distances = []
K = range(1, 300)
for k in K:
    km = clus.KMeans(n_clusters=k)
    km = km.fit(data_transformed)
    Sum_of_squared_distances.append(km.inertia_)

In [ ]:
plt.figure(figsize = (15, 8))
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlim(0,100)
plt.xlabel('Number of Clusters', size = 15)
plt.ylabel('Sum_of_squared_distances', size = 15)
plt.title('Elbow Method For Optimal k-clusters', size = 15)
plt.show()

### K-means clustering

In [9]:
# vector_df.head()
# plt.figure(figsize = (14, 8))
# sn.scatterplot(x='Vector_value',y='Index',data=vector_df)
# plt.xlabel('Highest  Vector Value', size = 15)
# plt.ylabel('Index of the highest vector value', size = 15)
# plt.grid()
# plt.show()

In [8]:
train_data = vector_df
kmean_Model = clus.KMeans(n_clusters = 300).fit(train_data)
# plt.figure(figsize = (14, 8))
# sn.scatterplot(x ='Vector_value',y ='Index', data =vector_df,hue=kmean_Model.labels_)
# plt.xlabel('Highest  Vector Value', size = 15)
# plt.ylabel('Index of the highest vector value', size = 15)
# plt.grid()
# plt.show()

In [9]:
### Getting which cluster a word with respect to context belong to
c_label = kmean_Model.labels_
label_df = pd.DataFrame()
for i in range(len(c_label)):
    label_df = label_df.append({'Cluster ID' : c_label[i]}, ignore_index = True)
print(label_df)

   Cluster ID
0         4.0
1         2.0
2         0.0
3         0.0
4         1.0
5         0.0
6         3.0
7         0.0
8         1.0
9         0.0


### Generating kmeans_output.txt

In [ ]:
fname_df = out_df['file_name']
word_df = out_df['wiki_word']
fname_word_df = pd.concat([fname_df, word_df], axis = 1)
final_df = pd.concat([fname_word_df, label_df], axis=1)
np.savetxt(r'C:/Users/ssund/Documents/adm_project1/kmeans_output.txt', final_df.values, fmt='%s, %s, %s')

In [10]:
### converting cluster labels to excel file
# label_df.to_excel('C:/Users/ssund/Documents/adm_project1/kmeans_label.xlsx', 'Sheet1', index = False)

### Dataframe to list

In [ ]:
km_output_df = pd.read_csv('C:/Users/ssund/Documents/adm_project1/kmeans_output.xlsx', names = ['file_name', 'wiki_word', 'cluster_ID'])
final_list = km_output_df['cluster_ID'].values.tolist()
# print(final_list)

In [9]:
# final_list = vector_df['Index'].values.tolist()
# print(final_list)

[849, 849, 849, 849, 849, 849, 849, 849, 849, 849, 1109, 849, 849, 1030, 205, 1114, 1109, 1109, 205, 1109, 1109, 1114, 1114, 205, 1109, 205, 1216, 1109, 1216, 205, 205, 1109, 1460, 1460, 1460, 1460, 1460, 1460, 346, 1109, 1059, 1109, 205, 1109, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 1109, 1109, 1109, 205, 205, 1109, 1109, 1109, 1109, 205, 1109, 1460, 205, 1109, 849, 1109, 205, 1109, 1109, 205, 1109, 1109, 205, 205, 205, 205, 1109, 205, 1109, 205, 1109, 205, 909, 205, 1109, 205, 205, 205, 205, 205, 1109, 205, 360, 360, 883, 883, 1109, 205, 1114, 1114, 1109, 205, 1109, 205, 1460, 205, 205, 1109, 1109, 1109, 205, 205, 1109, 205, 205, 205, 1114, 205, 205, 1460, 1109, 1460, 205, 1109, 1360, 1360, 205, 205, 205, 205, 1030, 1109, 205, 1030, 1030, 1109, 1460, 205, 1059, 849, 1109, 1109, 1109, 1109, 1360, 1109, 205, 1109, 1114, 1109, 205, 205, 1109, 1114, 1109, 1460, 1114, 1109, 883, 576, 1109, 205, 1216, 1216, 205, 205, 205, 205, 883, 1216, 1109, 1114, 1109, 205, 1460

### To Find the groups with most number of wiki_words

In [10]:
print(Counter(final_list).most_common(15))

[(205, 189666), (1109, 52502), (1114, 16247), (1460, 13542), (849, 11760), (1216, 8706), (346, 8122), (1030, 6651), (1062, 5725), (848, 4008), (1059, 2574), (931, 2529), (835, 2411), (909, 2205), (883, 1973)]
